In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy.stats import norm

import os
import time
import itertools

from glove.model import *

In [2]:
# import file names
files = os.listdir("data/")
files = [f for f in files if "processed" in f]
files

['EXP0019_MS001_processed.csv',
 'EXP0019_MS014_processed.csv',
 'EXP0019_MS008_processed.csv',
 'EXP0019_DSM27147_processed.csv']

# fit gLV models

In [3]:
exp_names = []
for file in files:
    # import data
    df = pd.read_csv(f"data/{file}")

    # determine species names 
    species = df.columns.values[2:]

    # instantiate gLV fit 
    model = gLV(species, df)

    # fit to data 
    model.fit()
    
    # list of parameter names 
    param_names = []
    for s1 in species:
        for s2 in species:
            param_names += [s1+"*"+s2]
    param_names = list(species) + param_names
    
    # plot parameter distribution
    n_species = len(species)
    Avec = model.params[n_species:]
    Aij_std = np.sqrt(np.diag(model.Ainv))[n_species:]

    plt.figure(figsize=(18,18))
    # set counter for parameter std. 
    k = 0

    for i in range(n_species):
        for j in range(n_species):
            plt.subplot(n_species, n_species, k+1)
            a = np.linspace(Avec[k]-np.std(Avec), Avec[k]+np.std(Avec))
            plt.plot(a, norm.pdf(a,Avec[k],Aij_std[k]))
            plt.axvline(x=0, c='k', alpha=.5)
            k += 1
            if j == 0:
                plt.ylabel(species[i], fontsize=18)
            if i == n_species-1:
                plt.xlabel(species[j], fontsize=18)
            #plt.xlim([-2,2])

    plt.suptitle(file.split("_")[0], fontsize=24)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig("figures/"+file.split("_")[1]+".pdf")
    plt.close()
    
    # compute Wald test for each parameter
    std_errors = np.sqrt(np.diag(model.Ainv))
    walds = model.params/std_errors
    wald_p_vals = 2*norm.cdf(-np.abs(walds))

    # save to df 
    df = pd.DataFrame()
    df["Param name"] = param_names
    df["Param value"] = model.params
    df["Param stdv"]  = np.sqrt(np.diag(model.Ainv))
    df["Param p-value"] = wald_p_vals
    for j, param_name in enumerate(param_names):
        df[param_name]  = model.Ainv[:, j]
    df.to_csv("params/"+file.split("_")[1]+".csv", index=False)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Total samples: 38, Updated regularization: 1.00e-03
Loss: 11.234, Residuals: -0.125
Loss: 5.450, Residuals: -0.004
Loss: 4.342, Residuals: -0.013
Loss: 3.542, Residuals: -0.020
Loss: 2.745, Residuals: -0.000
Loss: 2.569, Residuals: 0.027
Loss: 2.306, Residuals: 0.001
Loss: 2.273, Residuals: 0.015
Loss: 2.216, Residuals: 0.004
Loss: 2.131, Residuals: -0.014
Loss: 2.129, Residuals: -0.016
Loss: 2.126, Residuals: -0.015
Loss: 2.121, Residuals: -0.013
Loss: 2.079, Residuals: -0.023
Loss: 2.076, Residuals: -0.016
Loss: 2.051, Residuals: -0.023
Loss: 2.031, Residuals: -0.019
Loss: 2.028, Residuals: -0.021
Loss: 2.022, Residuals: -0.023
Loss: 2.012, Residuals: -0.028
Loss: 2.012, Residuals: -0.025
Loss: 2.003, Residuals: -0.029
Loss: 1.998, Residuals: -0.031
Loss: 1.998, Residuals: -0.030
Loss: 1.995, Residuals: -0.032
Loss: 1.989, Residuals: -0.035
Loss: 1.989, Residuals: -0.036
Evidence -221.400
Updating hyper-parameters...
Total samples: 38, Updated regularization: 2.07e-02
Loss: 11.104, R

Total samples: 31, Updated regularization: 1.82e+00
Loss: 121.271, Residuals: -0.004
Loss: 121.230, Residuals: -0.006
Loss: 121.220, Residuals: -0.007
Loss: 121.216, Residuals: -0.006
Loss: 121.209, Residuals: -0.007
Loss: 121.204, Residuals: -0.007
Loss: 121.204, Residuals: -0.007
Evidence 415.856
Updating hyper-parameters...
Total samples: 31, Updated regularization: 1.79e+00
Loss: 121.723, Residuals: -0.007
Evidence 416.219
Pass count  1
Total samples: 38, Updated regularization: 1.00e-03
Loss: 11.280, Residuals: -0.122
Loss: 5.382, Residuals: 0.005
Loss: 4.297, Residuals: -0.010
Loss: 3.550, Residuals: -0.030
Loss: 2.734, Residuals: -0.001
Loss: 2.548, Residuals: 0.017
Loss: 2.284, Residuals: -0.005
Loss: 2.268, Residuals: 0.012
Loss: 2.238, Residuals: 0.008
Loss: 2.185, Residuals: 0.000
Loss: 2.104, Residuals: -0.013
Loss: 2.103, Residuals: -0.015
Loss: 2.096, Residuals: -0.010
Loss: 2.082, Residuals: -0.013
Loss: 2.059, Residuals: -0.018
Loss: 2.053, Residuals: -0.001
Loss: 2.042

Loss: 150.378, Residuals: 0.009
Loss: 150.375, Residuals: 0.009
Evidence 487.447
Pass count  1
